# The Request

Observations every 12 hours for each of the patients in the cohort throughout the month of June 2000. For each of these observations, we need the following data
● The ratio of discharged patients over the previous 4 hours to the number of nurses in the patient’s unit
● The ratio of nurses to patients in the patient’s unit
● The patient’s latest Tropinin lab value (confirm codes)
● The patient’s latest B-type natriuretic peptide (BNP) lab value (LOINC 33762-6)

# The Deliverable
Long DF with one row per patient per observation

In [125]:
import pandas as pd
import sqlite3
import datetime

# Set up the database connection
con = sqlite3.connect("data/DE_Challenge_DB.sqlite")

# build scaffolding for the snapshots
scaffold = pd.date_range(start='2000-06-01 06:59:00', end='2000-06-30 23:59:00', freq='12h')

## Building out the Unit, Population and Staffing Information

Key measures that must accompany patient data consider the overall environment of the hospital at the snapshot times of 6:59 AM and 6:59 PM every day during the study period.  We start with date scaffolding that will help us gather information for each date/time of interst.  Here, we gather information that will help us understand
* where each patient is at the time of each of the snapshots (`snap_df`),
* who is on the nursing team for each patient at the snapshot time (`care_df`) and
* counts of discharges for each patient at the snapshot time (`discharge_df`)

In [126]:
snapdata = []
careteam = []
discharges = []

# collect data at each of the given snapshot points
for snapdate in scaffold:
    # where is the patient right now?
    patient_location = pd.read_sql_query(f"""
            SELECT SUBJECT_ID,
            CURR_CAREUNIT as UNIT
            FROM TRANSFERS WHERE INTIME <= '{snapdate}'
            AND OUTTIME > '{snapdate}'
        """, con = con)
    patient_location['snapshot_time'] = snapdate
    snapdata.append(patient_location)

    # who is the nursing team for each patient right now?
    care_detail = pd.read_sql_query(f"""
            SELECT SUBJECT_ID, tt.CGID, LABEL, DESCRIPTION
            FROM TREATMENT_TEAM tt
            LEFT JOIN CARE_GIVERS cg on tt.CGID = cg.CGID
            WHERE tt.STARTTIME <= '{snapdate}'
            AND tt.ENDTIME > '{snapdate}'
            AND lower(LABEL) = 'rn' OR lower(DESCRIPTION) = 'rn'
        """, con = con)
    care_detail['snapshot_time'] = snapdate
    careteam.append(care_detail)

    # how many patients were discharged from each unit up to four hours before right now?
    discharge_count = pd.read_sql_query(f"""
            SELECT PREV_CAREUNIT as UNIT,
            count(distinct(SUBJECT_ID)) as DISCHARGE_COUNT
            FROM TRANSFERS
            WHERE INTIME > '{snapdate - datetime.timedelta(hours = 4)}'
              AND INTIME <= '{snapdate}'
             AND EVENTTYPE = 'discharge'
            GROUP BY PREV_CAREUNIT
        """, con = con)
    discharge_count['snapshot_time'] = snapdate
    discharges.append(discharge_count)

We generate pandas dataframes from the gathered information

In [127]:
snap_df = pd.concat(snapdata)
care_df = pd.concat(careteam)
discharge_df = pd.concat(discharges).groupby(['snapshot_time','UNIT'],dropna=False).sum()

Here, we have a df showing where each patient was at a given time.

In [128]:
snap_df.head()

,SUBJECT_ID,UNIT,snapshot_time
0,124,None,2000-06-01 06:59:00
1,6,SICU,2000-06-01 06:59:00
2,109,None,2000-06-01 06:59:00
3,110,NWARD,2000-06-01 06:59:00
4,223,None,2000-06-01 06:59:00


Here, we have one row per patient per RN per careteam

In [129]:
care_df.head()

,SUBJECT_ID,CGID,LABEL,DESCRIPTION,snapshot_time
0,30593,19891.0,RN,RN,2000-06-01 06:59:00
1,30593,17735.0,RN,RN,2000-06-01 06:59:00
2,30593,15526.0,RN,RN,2000-06-01 06:59:00
3,30300,20808.0,RN,RN,2000-06-01 06:59:00
4,30593,20063.0,RN,RN,2000-06-01 06:59:00


For the sake of working data, we'll keep the discharge table in long format but here is an overview of it pivoted out by Unit.

In [130]:
discharge_df.reset_index().pivot(index ='snapshot_time', columns='UNIT', values = 'DISCHARGE_COUNT').fillna(0)

UNIT,NaN,CCU,CSRU,MICU,NICU,NWARD,SICU,TSICU
snapshot_time,,,,,,,,
2000-06-01 06:59:00,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2000-06-01 18:59:00,58.0,3.0,0.0,3.0,4.0,4.0,1.0,0.0
2000-06-02 06:59:00,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2000-06-02 18:59:00,49.0,3.0,0.0,8.0,4.0,1.0,2.0,1.0
2000-06-03 06:59:00,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2000-06-03 18:59:00,64.0,2.0,1.0,7.0,4.0,5.0,1.0,1.0
2000-06-04 06:59:00,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0
2000-06-04 18:59:00,58.0,0.0,2.0,3.0,3.0,1.0,1.0,2.0
2000-06-05 18:59:00,59.0,1.0,0.0,5.0,6.0,2.0,1.0,0.0


In [131]:
discharge_df.head()

DISCHARGE_COUNT
snapshot_time       UNIT                 
2000-06-01 06:59:00 MICU                5
                    NaN                 1
2000-06-01 18:59:00 CCU                 3
                    MICU                3
                    NICU                4

The Unit Population is a rollup of distinct SUBJECT_IDs in a given unit at each snapshot time.

In [132]:
unit_population = snap_df.groupby(['snapshot_time','UNIT'],dropna=False).nunique().rename(columns={'SUBJECT_ID':'PATIENT_COUNT'})

For review, here is the information pivoted by UNIT. For the actual patient analysis, we'll keep the `unit_population` table in long form.

In [133]:
unit_population.reset_index().pivot(index ='snapshot_time', columns='UNIT', values = 'PATIENT_COUNT').fillna(0)

UNIT,NaN,CCU,CSRU,MICU,NICU,NWARD,SICU,TSICU
snapshot_time,,,,,,,,
2000-06-01 06:59:00,816,67,85,225,220,46,129,84
2000-06-01 18:59:00,766,63,93,204,213,38,124,74
2000-06-02 06:59:00,802,68,91,197,220,43,133,75
2000-06-02 18:59:00,760,66,86,188,221,34,128,76
2000-06-03 06:59:00,803,65,86,193,220,39,126,76
2000-06-03 18:59:00,772,53,90,191,222,37,119,72
2000-06-04 06:59:00,813,59,88,211,224,40,125,73
2000-06-04 18:59:00,748,51,85,218,224,43,127,74
2000-06-05 06:59:00,791,53,83,234,229,50,140,77


In [134]:
unit_population.head()

PATIENT_COUNT
snapshot_time       UNIT                
2000-06-01 06:59:00 CCU               67
                    CSRU              85
                    MICU             225
                    NICU             220
                    NWARD             46

Now we can work backwards, since we know who was in each unit at a given time and work out how many unique RNs were associated with the unit at that time

In [135]:
caregiver_population = snap_df.set_index(['SUBJECT_ID', 'snapshot_time'])\
    .join(care_df.set_index(['SUBJECT_ID', 'snapshot_time']))\
    .dropna(subset = ['CGID'], axis=0)\
    .reset_index()\
    .drop(columns=['SUBJECT_ID','LABEL', 'DESCRIPTION'])\
    .groupby(['snapshot_time','UNIT'], dropna = False)\
    .nunique().rename(columns={'CGID':'TEAM_COUNT'})
caregiver_population.head()

TEAM_COUNT
snapshot_time       UNIT             
2000-06-01 06:59:00 CCU           119
                    CSRU          143
                    MICU          287
                    NICU           90
                    NWARD           8

One more time, just showing this wide but will work with the data long.

In [136]:
caregiver_population.reset_index().pivot(index ='snapshot_time', columns='UNIT', values = 'TEAM_COUNT').fillna(0)

UNIT,NaN,CCU,CSRU,MICU,NICU,NWARD,SICU,TSICU
snapshot_time,,,,,,,,
2000-06-01 06:59:00,353,119,143,287,90,8,207,154
2000-06-01 18:59:00,357,121,161,271,83,6,205,158
2000-06-02 06:59:00,354,121,164,266,84,4,193,150
2000-06-02 18:59:00,351,103,158,270,70,5,192,159
2000-06-03 06:59:00,350,101,164,286,87,6,195,154
2000-06-03 18:59:00,348,100,160,268,83,9,187,157
2000-06-04 06:59:00,348,115,163,281,89,4,183,162
2000-06-04 18:59:00,354,110,168,266,74,6,193,163
2000-06-05 06:59:00,355,101,167,277,89,7,180,168


In [137]:
unit_measures = pd.concat([unit_population, discharge_df, caregiver_population], axis = 1).fillna(0)
unit_measures['nurse_discharge_ratio'] = unit_measures.DISCHARGE_COUNT / unit_measures.PATIENT_COUNT
unit_measures

PATIENT_COUNT  DISCHARGE_COUNT  TEAM_COUNT  \
snapshot_time       UNIT                                                
2000-06-01 06:59:00 CCU               67              0.0         119   
                    CSRU              85              0.0         143   
                    MICU             225              5.0         287   
                    NICU             220              0.0          90   
                    NWARD             46              0.0           8   
...                                  ...              ...         ...   
2000-06-28 06:59:00 NaN              784              1.0         351   
2000-06-28 18:59:00 NaN              758             55.0         355   
2000-06-29 18:59:00 NaN              764             57.0         355   
2000-06-30 06:59:00 NaN              785              1.0         358   
2000-06-30 18:59:00 NaN              744             48.0         358   

                           nurse_discharge_ratio  
snapshot_time       UNIT                          
2000-06-01 06:59:00 CCU                 0.000000  
                    CSRU                0.000000  
                    MICU                0.022222  
                    NICU                0.000000  
                    NWARD               0.000000  
...                                          ...  
2000-06-28 06:59:00 NaN                 0.001276  
2000-06-28 18:59:00 NaN                 0.072559  
2000-06-29 18:59:00 NaN                 0.074607  
2000-06-30 06:59:00 NaN                 0.001274  
2000-06-30 18:59:00 NaN                 0.064516  

[522 rows x 4 columns]

## Building out the cohort's test result data

Now, we go get all the patients in our cohort:

In [138]:
patient_cohort = pd.read_sql_query("""
        SELECT DISTINCT june_admits.SUBJECT_ID,
                june_admits.DOB,
                june_admits.GENDER -- 381 rows
        FROM (SELECT DISTINCT T.SUBJECT_ID,
                              HADM_ID,
                              INTIME,
                              pts.GENDER,
                              pts.DOB
              FROM TRANSFERS T
                INNER JOIN (
                              SELECT SUBJECT_ID, GENDER, min(DOB) as DOB
                              FROM PATIENTS
                              WHERE EXPIRE_FLAG = 0
                              group by SUBJECT_ID, GENDER
                          ) pts
                                   on T.SUBJECT_ID = pts.SUBJECT_ID
                          WHERE T.EVENTTYPE = 'admit'
                            AND INTIME BETWEEN '2000-06-01' AND '2000-07-01'
                            AND (julianday(T.INTIME) - julianday(pts.DOB)) / 365 > 18) june_admits -- 2402 pts
        INNER JOIN
             (SELECT distinct SUBJECT_ID, HADM_ID
                  FROM LABS
                           INNER JOIN ICD_LABS ON LABS.ITEMID = ICD_LABS.ITEMID
                      AND ICD_LABS.LOINC_CODE in ('6598-7', '10839-9')
                      and LABS.VALUENUM is not NULL) tropo -- 1784 pts (of any age, any admit)
                 ON june_admits.SUBJECT_ID = tropo.SUBJECT_ID
                     AND june_admits.HADM_ID = tropo.HADM_ID
        INNER JOIN
             (SELECT distinct SUBJECT_ID, HADM_ID
              from DIAGNOSES
              WHERE (SUBSTR(DIAGNOSES.ICD9_CODE, 1, 3) like '42%')) dx -- 20,000 pts (of any age, any admit)
               ON june_admits.SUBJECT_ID = dx.SUBJECT_ID AND june_admits.HADM_ID = dx.HADM_ID;
            """, con = con)

In [139]:
total_patients = con.execute("SELECT COUNT(DISTINCT SUBJECT_ID) from PATIENTS").fetchone()[0]
print(f'''Total Number of Patients in Dataset: {total_patients}. Patient cohort size: {len(patient_cohort)}
Cohort represents {len(patient_cohort)/total_patients:.1%} of all patients in dataset.''')

Total Number of Patients in Dataset: 46488. Patient cohort size: 374
Cohort represents 0.8% of all patients in dataset.


## Get the most recent results from selected labs before a specific threshold date

In [140]:
def collect_latest_measure(lab_df, pt, threshold_date):
    lab_df = lab_df.loc[lab_df['SUBJECT_ID']==pt]
    # We want to collect a row when there's no data so we fully populate the grid of patient/scaffolding
    # We can always drop the information later
    # So we send back None as a signal for the calling code to build a dummy row
    if len(lab_df) == 0:
        return None
    else:
        try:
            latest_index = lab_df.loc[lab_df['CHARTTIME'] < threshold_date]['CHARTTIME'].idxmax()
            return dict(lab_df.loc[latest_index])
        except ValueError:
            return None


In [141]:
cohort_labs = []
tests = {'tropinin': ['6598-7', '10839-9'],
         'BNP': ['33762-6', '33762-6']} # hackery, I know. makes it possible to swap in 'tuple(tests[foo])' in the query

for test in tests:
    patient_labs = pd.read_sql_query(f"""SELECT LABS.SUBJECT_ID, CHARTTIME, VALUENUM, VALUEUOM, LABEL FROM LABS
                       INNER JOIN ICD_LABS ON LABS.ITEMID = ICD_LABS.ITEMID AND ICD_LABS.LOINC_CODE in {tuple(tests[test])}
                       WHERE SUBJECT_ID IN {tuple(patient_cohort['SUBJECT_ID'])}
                       AND LABS.VALUENUM IS NOT NULL""", con = con, parse_dates=['CHARTTIME'])
    patient_labs['test_family'] = test
    for patient in patient_cohort['SUBJECT_ID']:
        for snapdate in scaffold:
            latest_test = collect_latest_measure(patient_labs, patient, snapdate)
            if latest_test is None:
                latest_test = {'SUBJECT_ID':patient,
                              'CHARTTIME': None,
                              'VALUENUM': None,
                              'VALUEUOM': None,
                              'LABEL': None,
                              'test_family': test}
            latest_test['snapshot_time'] = snapdate
            cohort_labs.append(latest_test)

This remarkable bit of index mangling helps us drop some NaNs out of the columns

In [145]:
patient_labs = pd.DataFrame(cohort_labs).pivot(index=['snapshot_time', 'SUBJECT_ID'],
                                columns=[ 'test_family', 'LABEL'],
                                values=['VALUENUM'])
patient_labs.columns = patient_labs.columns.to_flat_index()
patient_labs = patient_labs.iloc[:,[1, 2, 4]]
patient_labs.columns = ['Troponin T', 'Troponin I', 'NTproBNP']

patient_labs

Troponin T  Troponin I  NTproBNP
snapshot_time       SUBJECT_ID                                  
2000-06-01 06:59:00 309                NaN         NaN       NaN
                    318                NaN         NaN       NaN
                    321                NaN         NaN       NaN
                    518                NaN         NaN       NaN
                    740                NaN         NaN       NaN
...                                    ...         ...       ...
2000-06-30 18:59:00 98649             0.05         NaN       NaN
                    99185             0.05         NaN       NaN
                    99358             3.31         NaN       NaN
                    99448             0.03         NaN       NaN
                    99714             0.01         NaN       NaN

[22440 rows x 3 columns]

## Build out Reporting Framework

Now, we build a framework of information with the units and subjects who are relevant at a given point in time, constrained to this cohort.

In [143]:
snap_cohort_df = snap_df[snap_df.SUBJECT_ID.isin(patient_cohort['SUBJECT_ID'])]\
    .dropna(subset='UNIT')\
    .set_index(['snapshot_time','UNIT'])

snap_cohort_df

SUBJECT_ID
snapshot_time       UNIT            
2000-06-01 06:59:00 MICU        4787
                    MICU        6440
                    MICU       12567
                    CCU        14522
                    SICU       29483
...                              ...
2000-06-30 18:59:00 CSRU       82685
                    CCU        99448
                    CSRU       99185
                    CSRU       94448
                    MICU       96324

[3067 rows x 1 columns]

... and then we graft on the unit data and the lab data

In [149]:
wide_data = snap_cohort_df.join(unit_measures, how = 'left')\
            .reset_index()\
            .set_index(['snapshot_time', 'SUBJECT_ID'])\
            .join(patient_labs, how = 'left')\
            .reset_index()

In [150]:
wide_data

,snapshot_time,SUBJECT_ID,UNIT,PATIENT_COUNT,DISCHARGE_COUNT,TEAM_COUNT,nurse_discharge_ratio,Troponin T,Troponin I,NTproBNP
0,2000-06-01 06:59:00,4787,MICU,225,5.0,287,0.022222,0.01,NaN,NaN
1,2000-06-01 06:59:00,6440,MICU,225,5.0,287,0.022222,0.46,NaN,NaN
2,2000-06-01 06:59:00,12567,MICU,225,5.0,287,0.022222,0.03,NaN,NaN
3,2000-06-01 06:59:00,14522,CCU,67,0.0,119,0.000000,0.11,NaN,NaN
4,2000-06-01 06:59:00,29483,SICU,129,0.0,207,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3062,2000-06-30 18:59:00,88472,CCU,62,2.0,118,0.032258,10.69,NaN,258.0
3063,2000-06-30 18:59:00,94448,CSRU,110,1.0,146,0.009091,0.13,NaN,NaN
3064,2000-06-30 18:59:00,96324,MICU,238,5.0,299,0.021008,0.05,NaN,NaN
3065,2000-06-30 18:59:00,99185,CSRU,110,1.0,146,0.009091,0.05,NaN,NaN


## Write it out

Finally, we clean up a bit and send this table to CSV

In [156]:
wide_data.columns = [x.lower().replace(' ', '_') for x in wide_data.columns]

In [158]:
wide_data.to_csv('de_challenge_retrospective.csv',index = False)